In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

### 0 - Read and process collected driver data
- Just including all drivers output from Linx here

In [ ]:
drivers <- fread( paste0(TMP_DIR, "drivers.csv"))

### Somatic Drivers 

- Read in prepped exome file

In [ ]:
somatic_exome <- 
fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/somatic_exome.csv") %>% 
 gb(chromosome, position, gene, tier) %>% 
 mu(ct = n(), hotspot = (tier == "HOTSPOT" | ct > 30))

- Hotspots

In [ ]:
hotspots <- 
somatic_exome %>% 
 fi(hotspot) %>% 
 tm(sampleId, chromosome, chromosomeBand = NA, gene, transcript, annotation, biallelic, method = "HOTSPOT")

- Biallelic

In [ ]:
biallelic <- 
somatic_exome %>% 
 fi(!hotspot, biallelic) %>% 
 tm(sampleId, chromosome, chromosomeBand = NA, gene, transcript, annotation, biallelic, method = "BIALLELIC")

- Inframe

In [ ]:
inframe <- 
somatic_exome %>% 
 fi(!hotspot, !biallelic, annotation %in% c("inframe_deletion", "inframe_insertion", "phased_inframe_insertion", "phased_inframe_deletion")) %>% 
 tm(sampleId, chromosome, chromosomeBand = NA, gene, transcript, annotation, biallelic, method = "INFRAME")

- Now DNDS! 

In [ ]:
dnds <- readRDS("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/dnds/dnds.Rds")

In [ ]:
##summary(dnds$nbreg)

In [ ]:
dnds$globaldnds

### Copy number drivers 

In [ ]:
purities <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/purities.csv")

In [ ]:
cn_gene <- fread("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/cn_gene.csv")

In [ ]:
amps <- 
cn_gene %>% 
 ij(purities %>% se(sampleId, ploidy), by = "sampleId") %>%
 fi(maxCopyNumber > (3*ploidy)) %>% 
 mu( method = ifelse(minCopyNumber > 3*ploidy, "AMP", "PARTIAL_AMP"))

In [ ]:
dels <- 
cn_gene %>% 
 ij( purities %>% se(sampleId, ploidy), by = "sampleId" ) %>%
 fi( minCopyNumber < .5 ) %>% 
 mu( method = "DEL")  

### Structual Variant Drivers

- Disruptions

In [ ]:
disruptions <- 
fread( paste0(TMP_DIR, "structural_variants/breakend.csv")) %>%
 fi(disruptive) %>% 
 tm(sampleId, chromosome, chromosomeBand = chrBand, gene, transcriptId, canonical, disruptive, method = "DISRUPTION") %>% 
 unique()